In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from commonregex import CommonRegex
import requests
import re

In [2]:
### Tie new charges to land registry

In [3]:
use_cols = ['Title Number', 'Tenure', 'Property Address', 'District', 'County',
            'Region', 'Postcode', 'Multiple Address Indicator', 'Price Paid',
            'Proprietor Name (1)', 'Company Registration No. (1)',
            'Proprietorship Category (1)', 'Proprietor (1) Address (1)',
            'Proprietor (1) Address (2)', 'Proprietor (1) Address (3)']

rorig = pd.read_csv('CCOD_COU_2019_11.csv', skipfooter=1, engine='python',
                    usecols=use_cols,
                    dtype={'Title Number': str,
                           'Company Registration No. (1)': str})

print(rorig.shape)
rorig.head(2)

(63745, 15)


,Title Number,Tenure,Property Address,District,County,Region,Postcode,Multiple Address Indicator,Price Paid,Proprietor Name (1),Company Registration No. (1),Proprietorship Category (1),Proprietor (1) Address (1),Proprietor (1) Address (2),Proprietor (1) Address (3)
0,NT72787,Freehold,"2 Horsham Drive, Top Valley (NG5 9AH)",CITY OF NOTTINGHAM,CITY OF NOTTINGHAM,EAST MIDLANDS,NG5 9AH,N,30000.0,PLACES FOR PEOPLE HOMES LIMITED,IP19447R,Industrial and Provident Society (Company),"80 Cheapside, London EC2V 6EE","4 The Pavilions, Ashton-on-Ribble, Preston PR2...",NaN
1,LT25408,Freehold,"98-100 Melton Road, Leicester (LE4 5EB)",LEICESTER,LEICESTER,EAST MIDLANDS,LE4 5EB,N,NaN,JADEMARK LIMITED,2847145,Limited Company or Public Limited Company,"98 - 100 Melton Road, Leicester",NaN,NaN


In [4]:
corig = pd.read_csv('charges_tempt.csv', dtype=str, header=0, index_col=None)

print(corig.shape)
corig.head(2)

(37480, 15)


,CompanyName,CompanyNumber,CompanyStatus,IncorporationDate,SICCode,status,persons_entitled,delivered_on,charge_number,classification.type,classification.description,particulars.type,particulars.description,RegAddress.District,RegAddress.PostCode
0,CHG-MERIDIAN UK LIMITED,01276016,Active,1976-09-06T00:00:00.000Z,"[{""code"":""64910"",""description"":""Financial leas...",outstanding,"[{""name"":""Bal Global Finance (UK) Limited""}]",2019-08-01T00:00:00.000Z,2529,charge-description,A registered charge,brief-description,A security charge dated 31/07/2019 between bal...,RUNNYMEDE,TW20 9AB
1,CHG-MERIDIAN UK LIMITED,01276016,Active,1976-09-06T00:00:00.000Z,"[{""code"":""64910"",""description"":""Financial leas...",outstanding,"[{""name"":""Bal Global Finance (UK) Limited""}]",2019-08-01T00:00:00.000Z,2528,charge-description,A registered charge,brief-description,A security charge dated 31/07/2019 between bal...,RUNNYMEDE,TW20 9AB


### Extract Data

In [5]:
def hasNumbers(inputString):
    return any(char.isdigit() for char in str(inputString))

In [6]:
cedit = corig.copy()

cedit.dropna(subset=['particulars.description'], inplace=True)
cedit['hasNum'] = cedit['particulars.description'].apply(hasNumbers)
cedit = cedit[cedit['hasNum'] == True]

print(cedit.shape)
cedit.tail(2)

(23197, 16)


,CompanyName,CompanyNumber,CompanyStatus,IncorporationDate,SICCode,status,persons_entitled,delivered_on,charge_number,classification.type,classification.description,particulars.type,particulars.description,RegAddress.District,RegAddress.PostCode,hasNum
37478,LITTLE WELLIES LIMITED,09014548,Active,2014-04-28T00:00:00.000Z,"[{""code"":""88910"",""description"":""Child day-care...",outstanding,"[{""name"":""National Westminster Bank PLC""}]",2019-12-02T00:00:00.000Z,4,charge-description,A registered charge,brief-description,"110 apperley road, bradford BD10 9SR.",BRADFORD,BD2 1NH,True
37479,LITTONDALE LIMITED,11756098,Active,2019-01-08T00:00:00.000Z,"[{""code"":""98000"",""description"":""Residents prop...",outstanding,"[{""name"":""Gatehouse Bank PLC""}]",2019-11-11T00:00:00.000Z,1,charge-description,A registered charge,brief-description,"4,6 & 8 church street, clowne, chesterfield, S...",OLDHAM,OL2 6NX,True


In [103]:
for i in cedit['particulars.description'].sample(5):
    print(i+ '\n')

Unit 28 harrogate business park wetherby road harrogate t/no NYK74577.

162 monifieth avenue, glasgow, G52 3DJ registered under title number GLA162695.

Spring bank tavern, 29 spring bank, hull, HU3 1AF. HS164980.

Fixed charge over freehold property at land on the east side of white hart road london title number TGL76861, for more details please refer to the instrument.

11-12 church street staveley chesterfield.



In [8]:
### create test descriptions
desc1 = 'The freehold property known as 26B piper close mansfield woodhouse mansfield NG19 7GG registered at h m land registry with title number NT463229.'
desc2 = '22 shedden park road, kelso ROX11811.'
desc3 = '243 plaistow road, london E15 3EU. Title number: EGL211881'
desc4 = 'The freehold land known as 128 cardiff road, reading, RG1 8PQ and 8 trafford road, reading, RG1 8JP registered at the land registry under title numbers BK124548 and BK98452.'

In [61]:
def hasAddress(inputString):
#     ret = re.findall('(\d{1,10}( \w+){1,10}( ( \w+){1,10})?( \w+){1,10}[,.](( \w+){1,10}(,)? [A-Z]{2}( [0-9]{5})?)?)', str(inputString))
    ret = re.match('(\d[0-9]{1,3} .+, [a-zA-Z]+[^\s])', str(inputString))
#     ret = re.findall('([A-Za-z0-9]\.\-\s\,])', str(inputString))

    if ret:
        return ret
    else:
        pass

In [62]:
### ok performance; needs improvement for termination
### will likely need multiple regexs for various forms of input
### desc1 is being missed because letter & number combination
print(hasAddress(desc1))
print(hasAddress(desc2))
print(hasAddress(desc3))
print(hasAddress(desc4))

None
<re.Match object; span=(0, 27), match='22 shedden park road, kelso'>
<re.Match object; span=(0, 25), match='243 plaistow road, london'>
None


In [11]:
### ok performance; unable to extract city + postcode, or n-grams greater than 3
print(CommonRegex(desc1).street_addresses)
print(CommonRegex(desc2).street_addresses)
print(CommonRegex(desc3).street_addresses)
print(CommonRegex(desc4).street_addresses)

[]
['22 shedden park road,']
['243 plaistow road,']
['128 cardiff road,', '1 8PQ and 8 trafford road,']


In [12]:
def hasPostcode(inputString):
    ret = re.findall(r'\b[A-Z]{1,2}[0-9][A-Z0-9]? [0-9][ABD-HJLNP-UW-Z]{2}\b', str(inputString))
    if len(ret) > 0:
        return ret
    else:
        pass

In [13]:
print(hasPostcode(desc1))
print(hasPostcode(desc2))
print(hasPostcode(desc3))
print(hasPostcode(desc4))

['NG19 7GG']
None
['E15 3EU']
['RG1 8PQ', 'RG1 8JP']


In [14]:
### hasPostcode as a somewhat indicator of the presence of an address
cedit['hasPostcode'] = cedit['particulars.description'].apply(hasPostcode)

cedit.loc[~cedit.hasPostcode.isnull()]['hasPostcode'].sample(5)

13756                     [SO16 5HA]
33715               [M1 7FG, M1 7FG]
1214            [CW10 9EU, CW10 9EU]
18115                      [SS0 9AJ]
13085    [EX4 3EH, LS1 6LQ, WR1 2DQ]
Name: hasPostcode, dtype: object

In [91]:
def biggerAddress(x):
    a_start = re.search('(\d[0-9]{1,3} .+, [a-zA-Z]+[^\s])', str(x))
    if a_start:
        a_start = a_start.span()[0]
    a_end = re.search(r'\b[A-Z]{1,2}[0-9][A-Z0-9]? [0-9][ABD-HJLNP-UW-Z]{2}\b', str(x))
    if a_end:
        a_end = a_end.span()[1]
    if a_start and a_end:
        return x[a_start:a_end]

In [92]:
biggerAddress(desc1)
biggerAddress(desc2)
biggerAddress(desc3)
biggerAddress(desc4)

'128 cardiff road, reading, RG1 8PQ'

In [93]:
print(desc1)
print(desc2)
print(desc3)
print(desc4)

The freehold property known as 26B piper close mansfield woodhouse mansfield NG19 7GG registered at h m land registry with title number NT463229.
22 shedden park road, kelso ROX11811.
243 plaistow road, london E15 3EU. Title number: EGL211881
The freehold land known as 128 cardiff road, reading, RG1 8PQ and 8 trafford road, reading, RG1 8JP registered at the land registry under title numbers BK124548 and BK98452.


In [54]:
test = re.match('(\d[0-9]{1,3} .+, [a-zA-Z]+[^\s])', desc4)

print(test)

None


In [49]:
desc4

'The freehold land known as 128 cardiff road, reading, RG1 8PQ and 8 trafford road, reading, RG1 8JP registered at the land registry under title numbers BK124548 and BK98452.'

In [67]:
ret = re.findall('(\d[0-9]{1,3} .+, [a-zA-Z]+[^\s])', desc4)
ret

['128 cardiff road, reading, RG1 8PQ and 8 trafford road, reading, RG1']

In [98]:
ret = re.finditer('(\d[0-9]{1,3} .+, [a-zA-Z]+[^\s])', desc4)

for i in ret:
    print(i)

<re.Match object; span=(27, 95), match='128 cardiff road, reading, RG1 8PQ and 8 trafford>


In [28]:
ret.span()[1]

27